### Workflow 

1. Use the labeled Dataset class that you wrote for milestone 2 and separate the images in the labeled folder into **train** and **validation** sets (having a validation set is especially important when training on a limited number of samples, since the danger of overfitting to training data is high).
1. Construct and train your supervised melanoma classifier. How does it perform on the test set?
1. Transfer learning is a popular machine learning (ML) approach when training data is scarce. Either choose a pre-trained model out of the ones available in the `torchvision.models` package, or train your own on a (large) dataset of your choice, and fine-tune it on your melanoma data. How does its performance compare to the fully supervised classifier that you trained from scratch in step 2?
1. When transfer learning does not prove to be sufficient, another option that we have is to take advantage of the unlabeled data by employing a semi-supervised ML strategy. Semi-supervised learning is a rapidly developing field, so the best place to look for the state-of-the-art methods is the famous arXiv.org, an open-access archive.  containing thousands of articles across numerous scientific disciplines, including computer science. Learning to work with the arXiv is an important skill for ML engineers involved in solving non trivial problems. **Spend some time to find at least 2-3 different approaches that have shown to be successful in semi-supervised ML for computer vision applications, and list them** along with the arXiv references that you found. It's fine to search arxiv.org and just read abstracts. See
  * https://arxiv.org/search/?query=semi+supervised+image+classification&searchtype=all



### Resources

* https://livebook.manning.com/book/grokking-deep-learning-for-computer-vision/chapter-6
* https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html
* https://livebook.manning.com/book/deep-learning-with-pytorch/chapter-2/81

In [1]:
url = 'https://liveproject-resources.s3.amazonaws.com/other/MelanomaDetection.zip'
pth = './data/MelanomaDetection.zip'
unp = './data/MelanomaDetection'
bs  = 24

In [2]:
import torch
import torchvision
from sklearn.model_selection import train_test_split
from torch.nn import BCELoss, BCEWithLogitsLoss, CrossEntropyLoss #not sure which I'd like to use. 
from torch.nn import Linear
from torch.optim import Adam
from torch.utils.data import Dataset, DataLoader, Subset, SubsetRandomSampler


In [3]:
%load_ext autoreload
%autoreload 2

from utils import *

### Create Training and Validation Sets

In [4]:
labeledSet = MILabeled(unp + '/labeled', ToTensor())

sd = 43
cs = len(labeledSet)
ci = range(0, cs)
train_idxs, test_idxs = train_test_split(ci, random_state=sd, train_size=0.75)

train_sampler = SubsetRandomSampler(train_idxs) #subset would work just fine here too I think. 
test_sampler  = SubsetRandomSampler(test_idxs)


In [5]:
labeled_ds = MILabeled(unp + '/labeled', ToTensor())
train_dl = DataLoader(labeled_ds, batch_size=bs, sampler=train_sampler)
test_dl = DataLoader(labeled_ds, batch_size=bs, sampler=test_sampler)

### Construct the Model

In [27]:
mod = torchvision.models.resnet18(False, True)

##Todo how to change n

In [34]:
print(mod)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [32]:
mod.fc

Linear(in_features=512, out_features=1000, bias=True)

In [36]:
#swap out the fully connected layer.  
mod.fc = Linear(512, 2)

In [42]:

criterion = CrossEntropyLoss()
optimizer = Adam(mod.parameters(), lr=1e-3)


### Write a Training Loop

In [1]:

def train_epoch(epoch, model, dataloader, criterion, optimizer):
    
    #set train to true
    model.train()
    
    loss_items = []
    
    #loop through batches in data loader
    for bi, (labels, images) in enumerate(dataloader):

        model.zero_grad()

        pred = model(batch)
       
        #left off here - do you think loss fn is correct?
        #https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html
        #https://discuss.pytorch.org/t/torchvision-models-dont-have-softmax-layer/18071/2 (Notes that this is why you use CrossEntropyLoss fn)
        loss = criterion(pred, labels)
        
        loss_items.append(loss.item())
        
        loss.backward()
        optimizer.step()
        
        #print batch results here
        
    #print epoch results here
    print('TRAIN | Epoch Loss: {}'.format(np.mean(loss_items)))



def valid_epoch(epoch, model, dataloader, criterion, bs):
    
    #set train to false
    model.eval()
    
    loss_items = []
    acc_list   = []
    
    #loop through batches in data loader
    for bi, (labels, images) in enumerate(dataloader):
    
        with torch.no_grad():
            pred = model(images)
    
    
        loss = criterion(pred, labels)
        loss_items.append(loss.item())
        
        #calculate the accuracy
        #https://discuss.pytorch.org/t/torchvision-models-dont-have-softmax-layer/18071/4
        plab = torch.argmax(pred, axis=1) 
        acc  = torch.eq(labels).sum().item() / bs
        
        assert(acc >= 0 and acc <= 1.0, "Your accuracy calculation is wrong.")
        
        #print batch results here
         
    
    #print epoch results here
    print('VALID | Epoch Loss: {}'.format(np.mean(loss_items)))

    




In [ ]:

for i in range(num_epochs):
    
    train_epoch(i, mod, train_dl, criterion, optimizer)
    test_epoch(i,  mod, test_dl,  criterion)

### Perform Transfer Learning

### Testing

In [12]:
dataiter = iter(test_dl)
labels, images = dataiter.next()
print(images.shape)
print(labels)

torch.Size([24, 3, 32, 32])
tensor([0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0])


In [21]:
for bi, b in enumerate(test_dl,0):
    labels, images = b
    print(labels)
    print(bi)
    break

tensor([0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0])
0


In [26]:
#messing with Subset
test_ds  = Subset(labeled_ds, test_idxs)
test_dl2 = DataLoader(test_ds, batch_size=bs)

dataiter = iter(test_dl2)
labels, images = dataiter.next()
print(images.shape)
print(labels)

torch.Size([24, 3, 32, 32])
tensor([0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1])
